In [15]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import pandas as pd
import numpy as np
import copy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import clean as clean

In [17]:
# Load the raw data
dt_raw = pd.read_csv('../data/TBI PUD 10-08-2013.csv')
dt_clean = clean.clean_data('../data/TBI PUD 10-08-2013.csv', remove_feats_after_ct=True, remove_TBI_rows_with_nan=True,threshold=0.5,rm_feats=True,remove_GCS_total_mismatch=True)

In [18]:
# Helper function to calculate False Negative Rate
def evaluate_fnr(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    fn = cm[1, 0]
    tp = cm[1, 1]
    return fn / (fn + tp)

In [19]:
# train and test load
X_train_balanced = np.load('../train_test/X_train_balanced.npy')
y_train_balanced = np.load('../train_test/y_train_balanced.npy')
X_test = np.load('../train_test/X_test.npy')
y_test = np.load('../train_test/y_test.npy')

In [20]:
# Train a Random Forest model with class weighting to address imbalance
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced")
rf_model.fit(X_train_balanced, y_train_balanced)

# Predictions
y_pred = rf_model.predict(X_test)
y_proba = rf_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# save the model in results folder
np.save('../results/randomforest.npy', y_proba)

In [21]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nROC AUC Score:")
print(roc_auc_score(y_test, y_proba))

Confusion Matrix:
[[8222   32]
 [  88   66]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      8254
         1.0       0.67      0.43      0.52       154

    accuracy                           0.99      8408
   macro avg       0.83      0.71      0.76      8408
weighted avg       0.98      0.99      0.98      8408


ROC AUC Score:
0.9769147741040156
